In [1]:
from bs4 import BeautifulSoup
from random import randint
from time import sleep
import re
import pandas as pd
import json
import math
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", -1)
# pd.set_option("display.float_format", "{:,.2f}".format)
pd.set_option("display.max_info_rows", 200)

In [2]:
# Get url link
sephora_url = "https://www.sephora.com"

We'll be able to scrape all products from the website by doing it per brand.  First, let's get a list of all brands and the corresponding url link using Requests and BeautifulSoup4.

In [240]:
chrome_path = "/Users/valmadrid/Downloads/chromedriver"
sephora_brands_url = "https://www.sephora.com/brands-list"

browser = webdriver.Chrome(executable_path = chrome_path)

browser.get(sephora_brands_url)
try:
        browser.find_element_by_xpath("//*[@id='MediaModal']/button").click()
except:
        pass
    
sleep(5)

elem = browser.find_element_by_tag_name("body")
no_of_pagedowns = 20
while no_of_pagedowns:
    elem.send_keys(Keys.PAGE_DOWN)
    sleep(0.5)
    no_of_pagedowns -= 1

sleep(5)

a_elems = browser.find_elements_by_xpath("//a")
brand_list=[]
for a in a_elems:
    brand_dict={}
    if (len(a.text)!=0) & (a.get_attribute("href").startswith("https://www.sephora.com/brand/")):
        brand_dict["brand_name"] = str(a.text).strip().replace(" NEW", "")
        brand_dict["brand_link"]= a.get_attribute("href")
        brand_list.append(brand_dict)

In [2]:
# df_brands = pd.DataFrame(brand_list)
# df_brands.to_csv("brands.csv", index=False)
df_brands = pd.read_csv("brands.csv")
df_brands.head(10)

,brand_name,brand_link
0,Acqua di Parma,https://www.sephora.com/brand/acqua-di-parma
1,adwoa beauty,https://www.sephora.com/brand/adwoa-beauty
2,AERIN,https://www.sephora.com/brand/aerin-perfume
3,Aether Beauty,https://www.sephora.com/brand/aether-beauty
4,Algenist,https://www.sephora.com/brand/algenist
5,Alpha-H,https://www.sephora.com/brand/alpha-h
6,alpyn beauty,https://www.sephora.com/brand/alpyn-beauty
7,ALTERNA Haircare,https://www.sephora.com/brand/alterna
8,amika,https://www.sephora.com/brand/amika
9,AMOREPACIFIC,https://www.sephora.com/brand/amorepacific


In [242]:
browser.close()

In [165]:
browser = webdriver.Chrome(executable_path = chrome_path)
test_brand_url = df_brands.brand_link.iloc[3]

browser.get(test_brand_url)
try:
        browser.find_element_by_xpath("//*[@id='MediaModal']/button").click()
except:
        pass
    
sleep(5)

elem = browser.find_element_by_tag_name("body")
no_of_pagedowns = 20
while no_of_pagedowns:
    elem.send_keys(Keys.PAGE_DOWN)
    sleep(0.5)
    no_of_pagedowns -= 1

In [166]:
soup = BeautifulSoup(browser.page_source)
browser.close()
soup.prettify()

'<html class="css-1esi5of no-touch" data-comp="Index " lang="en">\n <head data-comp="Head ">\n  <script src="//s.thebrighttag.com/tag?site=ky5EioO&amp;H=-1tr13d&amp;referrer=https%3A%2F%2Fwww.sephora.com%2Fbrand%2Faether-beauty&amp;mode=v2&amp;cf=7465926" type="text/javascript">\n  </script>\n  <script async="" src="https://s0.ipstatp.com/ad/business/track-log.js" type="text/javascript">\n  </script>\n  <script async="" src="https://s.pinimg.com/ct/lib/main.b3497531.js">\n  </script>\n  <script async="" src="https://s0.ipstatp.com/ad/business/track-log.js" type="text/javascript">\n  </script>\n  <script src="//s.thebrighttag.com/tag?site=ky5EioO&amp;H=-1tr13d&amp;referrer=https%3A%2F%2Fwww.sephora.com%2Fbrand%2Faether-beauty&amp;mode=v2&amp;cf=7465904" type="text/javascript">\n  </script>\n  <script src="//s.thebrighttag.com/tag?site=ky5EioO&amp;H=-1tr13d&amp;referrer=https%3A%2F%2Fwww.sephora.com%2Fbrand%2Faether-beauty&amp;mode=v2&amp;cf=7465920" type="text/javascript">\n  </script>\

In [168]:
soup.find_all(href=re.compile("/product/"))

[<a aria-label="Aether Beauty Solstice Eyeshadow Palette" class="css-ix8km1" data-comp="ProductItem " data-uid="P444401 2189975" href="/product/summer-solstice-eyeshadow-palette-P444401?icid2=products grid:p444401"><div class="css-79elbk"><div class="css-z3rgf2"><div class="css-1ymsisj" data-comp="ProductImage Box "><div class="css-1dn9kin"></div><img alt="Aether Beauty - Solstice Eyeshadow Palette Clean at Sephora Clean at Sephora" class="css-j3e40p" data-comp="Box " src="/productimages/sku/s2189975-main-zoom.jpg?pb=2020-03-sephora-clean-2019&amp;imwidth=162" srcset="/productimages/sku/s2189975-main-zoom.jpg?pb=2020-03-sephora-clean-2019&amp;imwidth=324 2x"/></div><button aria-label="More info on Aether Beauty Solstice Eyeshadow Palette" class="css-1g0g0sz" data-comp="ProductQuicklook " type="button">Quick Look</button><div class="css-1ifgyzz" style="opacity:0"><button aria-label="Sign in to love Aether Beauty Solstice Eyeshadow Palette" class="css-3x6bwe" data-at="unloved" data-comp=

In [169]:
soup.find_all(href=re.compile("/product/"))[0].attrs

{'class': ['css-ix8km1'],
 'aria-label': 'Aether Beauty Solstice Eyeshadow Palette',
 'href': '/product/summer-solstice-eyeshadow-palette-P444401?icid2=products grid:p444401',
 'data-comp': 'ProductItem ',
 'data-uid': 'P444401 2189975'}

In [171]:
soup.find_all(href=re.compile("/product/"))[0].find(attrs={"data-at": re.compile("sku_item_price_list")}).get_text(strip=True)

'$58.00'

In [175]:
soup.find_all(href=re.compile("/product/"))[0].find(attrs={"src": re.compile("/productimages/")}).get("src")

'/productimages/sku/s2189975-main-zoom.jpg?pb=2020-03-sephora-clean-2019&imwidth=162'

In [177]:
soup.find_all(href=re.compile("/product/"))[0].find(attrs={"data-at": re.compile("sku_item_name")}).get_text(strip=True)

'Solstice Eyeshadow Palette'

In [3]:
def get_page(url):
    chrome_path = "/Users/valmadrid/Downloads/chromedriver"
    browser = webdriver.Chrome(executable_path=chrome_path)
    
    browser.get(url)
    
    sleep(randint(7, 10))
    
    try:
        browser.find_element_by_xpath("//*[@id='MediaModal']/button").click()
    except:
        pass
    
    sleep(randint(7, 10))
   
    elem = browser.find_element_by_tag_name("body")
    no_of_pagedowns = 30
    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        sleep(1)
        no_of_pagedowns -= 1
        
    sleep(5)
    
    soup = BeautifulSoup(browser.page_source)
    browser.quit()
    return soup


def get_products(soup):
    sephora_url = "https://www.sephora.com"
    product_list = []
    if len(soup.find_all("a", href=re.compile("/product/"))):
        for tag in soup.find_all("a", href=re.compile("/product/")):
            product_dict = {}
            product_dict["brand_name"] = tag.find(
                    attrs={
                        "data-at": re.compile("sku_item_brand")
                    }).get_text(strip=True)
            try:
                product_dict["product_ref"] = tag.attrs["data-uid"].split()[0]
            except:
                product_dict["product_ref"] = np.nan
            try:
                product_dict["product_item"] = tag.attrs["data-uid"].split(
                )[-1]
            except:
                product_dict["product_item"] = np.nan
            try:
                product_dict["product_name"] = tag.find(
                    attrs={
                        "data-at": re.compile("sku_item_name")
                    }).get_text(strip=True)
            except:
                product_dict["product_name"] = np.nan
            try:
                product_dict["product_price"] = tag.find(
                    attrs={
                        "data-at": re.compile("sku_item_price_list")
                    }).get_text(strip=True)
            except:
                product_dict["product_price"] = np.nan
            try:
                product_dict["product_image"] = sephora_url + tag.find(
                    attrs={
                        "src": re.compile("/productimages/")
                    }).get("src")
            except:
                product_dict["product_image"] = np.nan
            try:
                product_dict["product_link"] = sephora_url + tag.attrs["href"]
            except:
                product_dict["product_link"] = np.nan
            product_list.append(product_dict)

    return product_list

In [4]:
# df_products = pd.DataFrame()
# for i in range(len(df_brands)):
#     product_list = []
#     for page in range(1, 5):
#         if page == 1:
#             full_url = df_brands.brand_link.iloc[i] + "?pageSize=300"
#         else:
#             full_url = df_brands.brand_link.iloc[
#                 i] + "?pageSize=300&currentPage=" + str(page)
#         soup = get_page(full_url)
#         products = get_products(soup)
#         product_list.extend(products)
#         if len(product_list) == 0 or len(products) < 300:
#             break
#     if i == 0:
#         df_products = pd.DataFrame(product_list)
#     else:
#         df_products = df_products.append(pd.DataFrame(product_list),
#                                          ignore_index=True)
#         sleep(3)
#     sleep(randint(2, 3))

In [244]:
df_products = pd.read_csv("products.csv")
df_products.head()

,brand_name,product_ref,product_item,product_name,product_price,product_image,product_link,b
0,Acqua di Parma,P443401,2218774.0,Blu Mediterraneo MINIATURE Set,$66.00,https://www.sephora.com/productimages/sku/s2218774-main-zoom.jpg?imwidth=162,https://www.sephora.com/product/blu-mediterraneo-minature-set-P443401?icid2=products grid:p443401,Acqua di Parma
1,Acqua di Parma,P307801,1417609.0,Fico di Amalfi,$30.00 - $180.00,https://www.sephora.com/productimages/sku/s1417609-main-zoom.jpg?imwidth=162,https://www.sephora.com/product/blu-mediterraneo-fico-di-amalfi-P307801?icid2=products grid:p307801,Acqua di Parma
2,Acqua di Parma,P163604,2044816.0,Colonia,$66.00 - $170.00,https://www.sephora.com/productimages/sku/s2044816-main-zoom.jpg?imwidth=162,https://www.sephora.com/product/colonia-P163604?icid2=products grid:p163604,Acqua di Parma
3,Acqua di Parma,P307804,1417617.0,Mirto di Panarea,$30.00 - $180.00,https://www.sephora.com/productimages/sku/s1417617-main-zoom.jpg?imwidth=162,https://www.sephora.com/product/blu-mediterraneo-mirto-di-panarea-P307804?icid2=products grid:p307804,Acqua di Parma
4,Acqua di Parma,P375388,1417567.0,Arancia di Capri,$30.00 - $180.00,https://www.sephora.com/productimages/sku/s1417567-main-zoom.jpg?imwidth=162,https://www.sephora.com/product/blu-mediterraneo-arancia-di-capri-P375388?icid2=products grid:p375388,Acqua di Parma


In [187]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5697 entries, 0 to 5696
Data columns (total 7 columns):
brand_name       object
product_ref      object
product_item     object
product_name     object
product_price    object
product_image    object
product_link     object
dtypes: object(7)
memory usage: 311.7+ KB


In [236]:
df_products["product_item"] = df_products["product_item"].map(lambda x: str(round(x)), na_action="ignore")

In [237]:
pattern = re.compile(r"P[0-9]+")
df_products.product_ref.fillna(df_products.product_link.map(lambda x: pattern.findall(x)[0]), inplace=True)
df_products[df_products.product_item.isna()]

,brand_name,product_ref,product_item,product_name,product_price,product_image,product_link
30,AERIN,P398823,NaN,Mini Amber Musk Eau de Parfum Travel Spray,$30.00,NaN,https://www.sephora.com/product/amber-musk-rollerball-P398823?icid2=products grid:p398823
31,AERIN,P433901,NaN,Ikat Jasmine Travel Spray,$30.00,NaN,https://www.sephora.com/product/ikat-jasmine-rollerball-P433901?icid2=products grid:p433901
32,AERIN,P388764,NaN,Gardenia Rattan,$130.00,NaN,https://www.sephora.com/product/gardenia-rattan-P388764?icid2=products grid:p388764
33,AERIN,P410846,NaN,Tangier Vanille,$130.00,NaN,https://www.sephora.com/product/tangier-vanille-P410846?icid2=products grid:p410846
34,AERIN,P442515,NaN,Aegea Blossom Eau de Parfum,$30.00 - $180.00,https://www.sephora.com/productimages/sku/s2192177-main-zoom.jpg?imwidth=162,https://www.sephora.com/product/aegea-blossom-eau-de-parfum-P442515?icid2=products grid:p442515
...,...,...,...,...,...,...,...
5552,Viktor&Rolf,P432809,NaN,Magic Sparkling Secret,$145.00,https://www.sephora.com/productimages/sku/s2109528-main-zoom.jpg?imwidth=162,https://www.sephora.com/product/magic-sparkling-secret-P432809?icid2=products grid:p432809
5565,Violet Voss,P425707,NaN,Ride or Die - PRO Eyeshadow Palette,$75.00,https://www.sephora.com/productimages/sku/s2020972-main-zoom.jpg?imwidth=162,https://www.sephora.com/product/pro-eyeshadow-palette-ride-or-die-P425707?icid2=products grid:p425707
5566,Violet Voss,P454982,NaN,Matte Vibes Eyeshadow Palette,$30.00,https://www.sephora.com/productimages/sku/s2329597-main-zoom.jpg?imwidth=162,https://www.sephora.com/product/violet-voss-matte-vibes-eyeshadow-palette-P454982?icid2=products grid:p454982
5567,Violet Voss,P449088,NaN,Bright Vibes Neon Pressed Pigment Palette,$30.00,https://www.sephora.com/productimages/sku/s2272102-main-zoom.jpg?imwidth=162,https://www.sephora.com/product/vibes-neon-pressed-pigments-P449088?icid2=products grid:p449088


In [238]:
df_products["product_price_low"] = df_products.product_price.map(lambda x: x.split("-")[0].strip(" $"))
df_products["product_price_high"] = df_products.product_price.map(lambda x: x.split("-")[-1].strip(" $"))
df_products

,brand_name,product_ref,product_item,product_name,product_price,product_image,product_link,product_price_low,product_price_high
0,Acqua di Parma,P443401,2218774,Blu Mediterraneo MINIATURE Set,$66.00,https://www.sephora.com/productimages/sku/s2218774-main-zoom.jpg?imwidth=162,https://www.sephora.com/product/blu-mediterraneo-minature-set-P443401?icid2=products grid:p443401,66.00,66.00
1,Acqua di Parma,P307801,1417609,Fico di Amalfi,$30.00 - $180.00,https://www.sephora.com/productimages/sku/s1417609-main-zoom.jpg?imwidth=162,https://www.sephora.com/product/blu-mediterraneo-fico-di-amalfi-P307801?icid2=products grid:p307801,30.00,180.00
2,Acqua di Parma,P163604,2044816,Colonia,$66.00 - $170.00,https://www.sephora.com/productimages/sku/s2044816-main-zoom.jpg?imwidth=162,https://www.sephora.com/product/colonia-P163604?icid2=products grid:p163604,66.00,170.00
3,Acqua di Parma,P307804,1417617,Mirto di Panarea,$30.00 - $180.00,https://www.sephora.com/productimages/sku/s1417617-main-zoom.jpg?imwidth=162,https://www.sephora.com/product/blu-mediterraneo-mirto-di-panarea-P307804?icid2=products grid:p307804,30.00,180.00
4,Acqua di Parma,P375388,1417567,Arancia di Capri,$30.00 - $180.00,https://www.sephora.com/productimages/sku/s1417567-main-zoom.jpg?imwidth=162,https://www.sephora.com/product/blu-mediterraneo-arancia-di-capri-P375388?icid2=products grid:p375388,30.00,180.00
...,...,...,...,...,...,...,...,...,...
5692,The 7 Virtues,P428702,2047777,Grapefruit Lime Eau de Parfum,$29.00 - $79.00,https://www.sephora.com/productimages/sku/s2047777-main-zoom.jpg?pb=2020-03-sephora-clean-2019&imwidth=162,https://www.sephora.com/product/grapefruit-lime-eau-de-parfum-P428702?icid2=products grid:p428702,29.00,79.00
5693,The 7 Virtues,P428705,2047678,Jasmine Neroli Eau de Parfum Rollerball,$29.00,https://www.sephora.com/productimages/sku/s2047678-main-zoom.jpg?pb=2020-03-sephora-clean-2019&imwidth=162,https://www.sephora.com/product/jasmine-neroli-eau-de-parfum-rollerball-P428705?icid2=products grid:p428705,29.00,29.00
5694,8Greens,P437988,2162444,8G Dietary Supplement,$14.00 - $78.00,https://www.sephora.com/productimages/sku/s2162444-main-zoom.jpg?imwidth=162,https://www.sephora.com/product/8greens-dietary-supplement-P437988?icid2=products grid:p437988,14.00,78.00
5695,8Greens,P452918,2298891,8Greens Gummies Dietary Supplement,$45.00,https://www.sephora.com/productimages/sku/s2298891-main-zoom.jpg?imwidth=162,https://www.sephora.com/product/8greens-8greens-gummies-dietary-supplement-P452918?icid2=products grid:p452918,45.00,45.00


In [239]:
df_products.to_csv("final_products.csv", index=False)